# 모델 로딩 from mlflow

----
* 다루는 내용
    * 모델 불러오기
    * 특정 버전, 최근 버전 불러오기
    * 운영 모델 불러오기
----

## 1. 환경준비

### (1) 라이브러리 Import

* 라이브러리 로딩

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import mlflow

### (2) 데이터 불러오기

* 카시트 판매량 데이터

|	변수명	|	설명	|	구분	|
|----|----|----|
|	**Sales** 	|	 **각 지역 판매량(단위 : 1000개)**	|	**Target**	|
|	CompPrice 	|	지역별 경쟁사 판매가격(달러)	|	feature	|
|	Income 	|	가구당 평균 소득액(1000달러)	|	feature	|
|	Advertising 	|	 각 지역, 회사의 광고 예산(1000달러)	|	feature	|
|	Population 	|	 지역 인구수(단위 : 1000명)	|	feature	|
|	Price 	|	 자사 지역별 판매가격(달러)	|	feature	|
|	ShelveLoc 	|	 진열상태(범주 : Bad, Medium, Good)	|	feature	|
|	Age 	|	 지역 인구의 평균 연령	|	feature	|
|	Education 	|	 교육수준(범주 : 10~18)	|	feature	|
|	Urban 	|	 매장이 도심에 있는지 여부(범주 : Yes, No)	|	feature	|
|	US 	|	 매장이 미국에 있는지 여부(범주 : Yes, No)	|	feature	|


* 데이터 경로 : https://raw.githubusercontent.com/DA4BAM/dataset/master/Carseats.csv

In [2]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/Carseats.csv'
carseat = pd.read_csv(path)
test = carseat.sample(5, random_state = 20)
test

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
10,9.01,121,78,9,150,100,Bad,26,10,No,Yes
382,4.95,121,28,19,315,121,Medium,66,14,Yes,Yes
374,9.44,131,47,7,90,118,Medium,47,12,Yes,Yes
196,4.10,130,28,6,410,133,Bad,72,16,Yes,Yes
47,4.38,126,98,0,173,108,Bad,55,16,Yes,No


## 2. 전처리 및 추적 준비

### (1) 데이터 전처리

* 데이터 준비

In [3]:
target = 'Sales'
x = test.drop(target, axis=1)

* 가변수화

In [4]:
cat_cols = {'ShelveLoc':['Bad','Medium','Good']
            , 'Education':[10,11,12,13,14,15,16,17,18]
            , 'US':['No','Yes']
            , 'Urban':['No','Yes']}

In [5]:
for var, values in cat_cols.items():
    x[var] = pd.Categorical(x[var], categories = values)

x = pd.get_dummies(x, columns = cat_cols.keys(), drop_first = True)    

In [6]:
x

,CompPrice,Income,Advertising,Population,Price,Age,ShelveLoc_Medium,ShelveLoc_Good,Education_11,Education_12,Education_13,Education_14,Education_15,Education_16,Education_17,Education_18,US_Yes,Urban_Yes
10,121,78,9,150,100,26,False,False,False,False,False,False,False,False,False,False,True,False
382,121,28,19,315,121,66,True,False,False,False,False,True,False,False,False,False,True,True
374,131,47,7,90,118,47,True,False,False,True,False,False,False,False,False,False,True,True
196,130,28,6,410,133,72,False,False,False,False,False,False,False,True,False,False,True,True
47,126,98,0,173,108,55,False,False,False,False,False,False,False,True,False,False,False,True


### (2) Tracking 준비
* mlflow uri를 setting 해야 모델 로딩도 가능합니다.

In [7]:
mlflow_uri = "sqlite:///mlflow.db"
mlflow.set_tracking_uri(mlflow_uri)

## 3. model loading

* 각 버전별 불러오기

In [8]:
# 버전으로 가져오기
model_uri = "models:/CarSeat/1" 
model1 = mlflow.sklearn.load_model(model_uri)

# 최신버전으로 가져오기
model_uri = "models:/CarSeat/latest" 
model2 = mlflow.sklearn.load_model(model_uri)

# 운영중인 버전 가져오기
model_uri = "models:/CarSeat/production" 
model3 = mlflow.sklearn.load_model(model_uri)

In [9]:
model1.predict(x)

array([8.0903, 6.9888, 7.0077, 4.1923, 5.8295])

In [10]:
model2.predict(x)

array([8.38283529, 6.66174308, 7.52758103, 2.66492697, 5.87916672])

In [11]:
model3.predict(x)

array([8.38283529, 6.66174308, 7.52758103, 2.66492697, 5.87916672])

- 모델이 다 다르니까 결과도 다 다르다.

## 4. 실습
* 실습을 해 봅시다.

In [ ]:
import joblib

### (1) 데이터 준비

* 데이터 로딩

In [ ]:
x_test = joblib.load('x_test.pkl')
x_test_s = joblib.load('x_test_s.pkl')
y_test = joblib.load('y_test.pkl')

| 칼럼명 | 설명 | | 값 의미 |  
|-----|-----|-----|-----|
|	Creditability	|	Creditability(Target)	|	향후 신용도	|	0 : Bad, 1 : Good	|
|	AccountBalance	|	Account Balance	|	은행잔고	|	1: No account, 2 : None (No balance), 3 : Some Balance	|
|	CreditDuration	|	Duration of Credit (month)	|	신청한 대출기간(월)	|	숫자	|
|	Payment	|	Payment Status of Previous Credit	|	과거 대출 납입 상태	|	0 : 연체, 1 : 기타신용, 2 : 완납, 3 : 정상 대출상환 중 |
|	Purpose	|	Purpose	|	신청한 대출목적	|	1 : New Car , 2 : Used Car , 3 : Furniture , 4 : TV , 5 : Appliances , 6 : Repair , 8 : Vacation , 9 :Retraining , 10 : Business , 0 : Other	|
|	CreditAmount	|	Credit Amount($)	|	신청한 대출금액	|		|
|	Employment	|	Length of current employment(Month)	|	현 직업 근무 기간	|	1: Unemployed,  2: <1 Year,  3: [1, 4),  4: [4, 7),  5: Above 7	|
|	SexMarital	|	Sex & Marital Status	|	성별 & 결혼상태	|	1: Male, Divorced, 2: Male, Single , 3: Male, Married/Widowed , 4: Female	|
|	CurrentAddress	|	Duration in Current address	|	현 거주지 거주기간	|	1: <1 Year , 2: [1, 4) , 3: [4, 7) , 4: Above 7	|
|	MostValuable	|	Most valuable available asset	|	가장 가치있는 자산	|	1: None , 2: Car , 3: Life Insurance , 4: Real Estate	|
|	Age	|	Age (years)	|	나이	|		|
|	AppartmentType	|	Type of apartment	|	주거환경	|	1: free apartment, 2: Rented, 3: Owned	|
|	NoCredits	|	No of Credits at this Bank	|	현재 총 대출 건수	|	1 : one, 2 : 2 ~ 3, 3 : 4 ~ 5, 4 : 6 ~	|
|	Occupation	|	Occupation	|	직업	|	1: Unemployed, unskilled, 2: Unskilled Permanent Resident, 3: Skilled, 4: Executive	|


### (2) mlflow ui 모델 관리
* 성능을 비교하고 가장 성능이 좋은 모델을 production으로 지정

### (3) production 모델 로딩

In [ ]:
model_uri = "  " 
model_credit = mlflow.

### (4) 테스트 데이터로 예측하기

In [ ]:
model_credit.predict(   )